## Imports

In [1]:
from src.ragtag.db.db_connection import DBConnection
from src.ragtag.backend.ragtag import RagTag

## Setup and methods def

In [2]:
class RagTagIntegration():
    def __init__(self):
        self.db_path = "test_database.db"
        
        self.ragtag = RagTag(DBConnection(self.db_path))

    def add_test_documents(self):
        documents = [
            "Python is a programming language with a simple syntax.",
            "To create a list in Python, use square brackets.",
            "Pancakes usually contain flour, eggs, and milk."
        ]
        return self.ragtag.add_documents_bulk(documents)


## Instantiate RAG Engine
### Add test documents

In [3]:
rageng = RagTagIntegration()
rageng.add_test_documents()

[9, 10, 11]

## Run a search

In [4]:
# Search for documents containing the word "Python"
search_results = rageng.ragtag.search_documents("Python")
print(f"Found {len(search_results)} documents matching 'Python':\n\n{search_results}")
print([rageng.ragtag.document_dao.read_document(doc[0]) for doc in search_results])

# Search for documents containing the word "Pancakes"
search_results = rageng.ragtag.search_documents("Pancakes")
print(f"Found {len(search_results)}  documents matching 'Pancakes':\n\n{search_results}")
print([rageng.ragtag.document_dao.read_document(doc[0]) for doc in search_results])


Found 2 documents matching 'Python':

[(9, 0.9515364944314393), (10, 0.9515364944314393)]
['Python is a programming language with a simple syntax.', 'To create a list in Python, use square brackets.']
Found 1  documents matching 'Pancakes':

[(11, 1.7301953284985192)]
['Pancakes usually contain flour, eggs, and milk.']
